# ***References***

- [1] https://openai.com/blog/function-calling-and-other-api-updates
- [2] https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb
- [3] https://rapidapi.com/weatherapi/api/weatherapi-com/
- [4] https://platform.openai.com/account/api-keys

In [51]:
import os
import json
os.getcwd()

'/config/workspace/research'

### ***Previous Ways***

#### ***Via Openai Python ChatCompletion class***


In [ ]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)



#### ***Via CURL command***


In [ ]:

curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-3.5-turbo",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."}, 
      {"role": "user", "content": "Hello!"}
      ]
  }'



#### ***Via requests module***


In [ ]:
import requests

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + OPENAI_API_KEY,
}
json_data = {
    "model": "gpt-3.5-turbo", 
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": "Hello!"}
        ]}
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=json_data,
)

### ***Weather Chatbot example***

In [52]:
import requests
from dotenv import load_dotenv
import json
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GPT_MODEL = "gpt-3.5-turbo-0613" # new model for function calling abilities

### ***Checkout the example***

In [ ]:
curl https://api.openai.com/v1/chat/completions -u :$OPENAI_API_KEY -H 'Content-Type: application/json' -d '{
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"}
  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}'
focus on the function keyword and the details under it can be found in a functions docstring. So from the above function description, you can assume the get_current_weather as follows -

def get_current_weather(location, unit):
    """Get the current weather in a given location

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        unit (str): ["celsius", "fahrenheit"]
    """
    pass

In [54]:
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + OPENAI_API_KEY,
    }
    json_data = {
        "model": model, 
        "messages": messages
        }
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )

        return response
    except Exception as e:
        print(f"\n>>>> Unable to generate ChatCompletion response")
        print(f"\n>>>> Exception: {e}")
        raise e

In [55]:
# simple use

sample_messages = [
    {"role": "system", "content": "you are a helpful assistant"},
    {"role": "user", "content": "Hi there"},
]

chat_response = chat_completion_request(messages=sample_messages)

chat_response

TypeError: can only concatenate str (not "NoneType") to str

In [56]:
import openai

def chat_completion_request(messages, functions=None, function_call=None, model="gpt-3.5-turbo"):
    OPENAI_API_KEY = "YOUR_API_KEY"  # Replace with your actual API key
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + OPENAI_API_KEY,
    }
    
    json_data = {
        "model": model, 
        "messages": messages
    }
    
    if functions is not None:
        json_data["functions"] = functions
    
    if function_call is not None:
        json_data["function_call"] = function_call
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=json_data)
    response_data = response.json()
    return response_data

sample_messages = [
    {"role": "system", "content": "you are a helpful assistant"},
    {"role": "user", "content": "Hi there"},
]

chat_response = chat_completion_request(messages=sample_messages)

print(chat_response)


{'error': {'message': 'Incorrect API key provided: YOUR_API_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


In [57]:
chat_response

{'error': {'message': 'Incorrect API key provided: YOUR_API_KEY. You can find your API key at https://platform.openai.com/account/api-keys.',
  'type': 'invalid_request_error',
  'param': None,
  'code': 'invalid_api_key'}}

In [58]:
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message

AttributeError: 'dict' object has no attribute 'json'

In [59]:
creator_name = "Yashwanth1119"

system_instruction = f"""
You are Weather bot created by {Yashwanth1119}, \
You first greet the customer, and you MUST introduce yourself as -
"Hi I am Weather bot created by {Yashwanth1119}! How may I assist you today?" \
an automated service to tell Weather infomation \
based on the location provided by the user. \
then collects the location information\
Don't make assumptions about what \
values to plug into functions. \
Ask for clarification if a user request is ambiguous.
You respond in a short, very conversational friendly style. \
"""

NameError: name 'Yashwanth1119' is not defined

In [60]:
messages = [
    {"role": "system", "content": system_instruction}
]

In [61]:
functions = [
    {
        "name": "get_current_weather", # name of your function
        "description": "Get the current weather", # description of your function
        "parameters": {
            "type": "object",
            "properties": { # 1st argument of your function
                "location": { # argument name
                    "type": "string", # argument type
                    "description": "The city and state, e.g. San Francisco, CA", # description of your function with example
                }
            },
            "required": ["location"], # required argument of your
        },
    }
]

In [62]:
def get_current_weather(location):
    RAPID_API_KEY = os.getenv("RAPID_API_KEY")
    try:
        url = "https://weatherapi-com.p.rapidapi.com/current.json"

        querystring = {"q": location}
        print(f"\n>>>> got the querystring as: {querystring}")
        headers = {
            "X-RapidAPI-Key": RAPID_API_KEY,
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        response_json = response.json()
        print(f"\n>>>> got the RAPID API response as: \n{response_json}")
        return response_json
    except Exception as e:
        raise e

In [63]:
user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)

In [64]:
chat_response


{'error': {'message': 'Incorrect API key provided: YOUR_API_KEY. You can find your API key at https://platform.openai.com/account/api-keys.',
  'type': 'invalid_request_error',
  'param': None,
  'code': 'invalid_api_key'}}

In [65]:
chat_response.json()


AttributeError: 'dict' object has no attribute 'json'

In [66]:
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message

AttributeError: 'dict' object has no attribute 'json'